In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, EfficientNetB0, resnet50, efficientnet
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------------
# 1. Load the Food-101 Dataset
# ---------------------------------------------------
# Create train/test split
(dataset_train, dataset_test), info = tfds.load(
    'food101',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes in Food-101
num_classes = info.features['label'].num_classes

# ---------------------------------------------------
# 2. Preprocess Data for ResNet and EfficientNet
# ---------------------------------------------------
IMG_SIZE = 224
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_resnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = resnet50.preprocess_input(image)
    return image, tf.one_hot(label, num_classes)

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = efficientnet.preprocess_input(image)
    return image, tf.one_hot(label, num_classes)

# Create tf.data pipelines
train_ds_resnet = (dataset_train
                   .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                   .batch(batch_size)
                   .prefetch(AUTOTUNE))

val_ds_resnet = (dataset_test
                 .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                 .batch(batch_size)
                 .prefetch(AUTOTUNE))

train_ds_eff = (dataset_train
                .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
                .batch(batch_size)
                .prefetch(AUTOTUNE))

val_ds_eff = (dataset_test
              .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
              .batch(batch_size)
              .prefetch(AUTOTUNE))

# ---------------------------------------------------
# 3. Define ResNet50 Model
# ---------------------------------------------------
def create_resnet_model(input_shape, num_classes):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = True

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="ResNet")
    return model

# ---------------------------------------------------
# 4. Define EfficientNetB0 Model
# ---------------------------------------------------
def create_efficientnet_model(input_shape, num_classes):
    base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = True

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="EfficientNet")
    return model

# ---------------------------------------------------
# 5. Compile and Train the Models
# ---------------------------------------------------
# ResNet50 Model
resnet_model = create_resnet_model((IMG_SIZE, IMG_SIZE, 3), num_classes)
resnet_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning ResNet50 on Food-101 ---")
history_resnet = resnet_model.fit(train_ds_resnet, epochs=3, validation_data=val_ds_resnet, verbose=1)

# EfficientNetB0 Model
eff_model = create_efficientnet_model((IMG_SIZE, IMG_SIZE, 3), num_classes)
eff_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning EfficientNetB0 on Food-101 ---")
history_eff = eff_model.fit(train_ds_eff, epochs=3, validation_data=val_ds_eff, verbose=1)

# Evaluate ResNet50
loss_resnet, acc_resnet = resnet_model.evaluate(val_ds_resnet, verbose=0)
print(f"ResNet50 Final Accuracy: {acc_resnet:.4f}")

# Evaluate EfficientNetB0
loss_eff, acc_eff = eff_model.evaluate(val_ds_eff, verbose=0)
print(f"EfficientNetB0 Final Accuracy: {acc_eff:.4f}")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=10000.0 (msgs/sec)
NotebookApp.rate_limit_window=1.0 (secs)



1894/1894 ━━━━━━━━━━━━━━━━━━━━ 744s 362ms/step - accuracy: 0.3888 - loss: 2.6600 - val_accuracy: 0.6454 - val_loss: 1.3952
Epoch 2/3
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 661s 349ms/step - accuracy: 0.7111 - loss: 1.0946 - val_accuracy: 0.6572 - val_loss: 1.4047
Epoch 3/3
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 663s 350ms/step - accuracy: 0.8495 - loss: 0.5523 - val_accuracy: 0.6580 - val_loss: 1.5169
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

--- Fine-Tuning EfficientNetB0 on Food-101 ---
Epoch 1/3
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 420s 184ms/step - accuracy: 0.2785 - loss: 3.2781 - val_accuracy: 0.6906 - val_loss: 1.2103
Epoch 2/3
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 293s 154ms/step - accuracy: 0.6665 - loss: 1.2891 - val_accuracy: 0.7351 - val_loss: 1.0086
Epoch 3/3
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 293s 155ms/step - accuracy: 0.7648 - loss: 0.8935 - val_accuracy: 0.7527 - val_loss: 0.9477
ResNet50 Final Accuracy: 0.6580
EfficientNetB0 Final Accuracy: 0.7527
